In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import nltk
import re
import spacy
from spacy.lang.en import English
from nltk.corpus import wordnet as wn
from nltk.stem.wordnet import WordNetLemmatizer
from gensim import corpora
import pickle
import gensim
import pyLDAvis.gensim
from gensim.models import LsiModel
from gensim.models.coherencemodel import CoherenceModel

import glob
from os import path
from PIL import Image



In [2]:
df = pd.read_csv('../data_processed/concatenated_data_cleaned_labeled.csv')

In [3]:
# Create English parser
#spacy.load('en')
parser = English()

def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            lda_tokens.append('URL')
        elif token.orth_.startswith('\n'):
            continue
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens

def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma
        print(lemma)
    
def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)

# List of stopwords to upload
en_stop = set(nltk.corpus.stopwords.words('english'))

desc_tokens = []

for index, row in df.iterrows():    
    tokens = tokenize(row['description'])
    #print(index)
    # Filters tokens to only words of length 4 or more
    tokens = [token for token in tokens if len(token) > 4]
    # Filters tokens that are in list of stopwords
    tokens = [token for token in tokens if token not in en_stop]
    # Lemmatizes tokens to their roots
    tokens = [get_lemma(token) for token in tokens]
    desc_tokens.append(tokens)

df['desc_tokens'] = desc_tokens

In [4]:
df.head()

,bin_role,company,description,location,title,url,desc_tokens
0,Data Engineer,Noom Inc.,"At Noom, we use scientifically proven methods...","New York, NY",Data Engineer,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,"[scientifically, prove, method, user, create, ..."
1,Data Scientist,Strategic Financial Solutions,Overview\nDo you love numbers and finding the...,"New York, NY 10018",Lead Data Scientist,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,"[overview, numbers, finding, story, numbers, t..."
2,Data Scientist,Averity,We are one of the world's premiere travel tec...,"New York, NY","Manager, Data Science",https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,"[world, premiere, travel, technology, company,..."
3,Data Scientist,Lenmar Consulting,Medical Data Scientist Must have at least 2 y...,"Woodcliff Lake, NJ",Medical Data Scientist,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,"[medical, scientist, least, years, clinical, r..."
4,Data Scientist,Memorial Sloan Kettering,"<h2 class=""jobSectionHeader"">Company Overview...","New York, NY 10017",Sr Strategic Consultant / Data Scientist (Pati...,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,"[class=""jobsectionheader"">company, overview, m..."


In [5]:
df.to_csv('../data_processed/concatenated_data_cleaned_labeled_preprocessed.csv', index=False)
